In [1]:
# Trick to autoreload modules automatically for development
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import time
import numpy as np
import matplotlib.pyplot as plt

from combined_fit import spectrum as sp
from combined_fit import mass 
from combined_fit.tensor import *
from combined_fit.gumbel import *
from combined_fit.xmax_distr import *


In [3]:
plt.rcParams.update({'font.size': 14,'legend.fontsize': 12}) #selecting font and size


Parameters choice at the source (gamma and logRcut), redshift range (zmin, zmax) that has to be read in the tensor and thresold energy, from which the deviance is performed


In [4]:
#Spectral parameters
logRcut = 18.25
gamma = -1.12
    
E_times_k= np.dot([0.05,  0.36,  0.40,  0.18, 0.01], 3.7150108e+46) # erg per solar mass

Chosing evolution of the source (SFR or flat at this stage) and computing weights for the injected spectra


In [5]:
#Evolution
S_z = Load_evol("SFR") #SFR evolution
#S_z =  lambda z: SFRd(1) #Flat evolution
    
w_R = lambda ZA, logR: Spectrum_Energy(ZA, logR, gamma, logRcut)
w_zR = lambda ZA, z, logR: w_R(ZA, logR)/dzdt(z)*S_z(z)

TypeError: Load_evol() takes 0 positional arguments but 1 was given

Initial mass fractions for the five injected masses

In [ ]:
model="Sibyll2.3d"
E_th = 18.75 # Compute the deviance from this energy


Upload the tensor provided in the folder "Tensor"

In [ ]:
#Evolution
S_z = Load_evol("SFR") #SFR evolution
#S_z =  lambda z: SFRd(1) #Flat evolution
    
w_R = lambda ZA, logR: Spectrum_Energy(ZA, logR, gamma, logRcut)
w_zR = lambda ZA, z, logR: w_R(ZA, logR)/dzdt(z)*S_z(z)

In [ ]:
    Tensor=[]
    Tensor = upload_Tensor()

Upload the experimental distribution

In [ ]:
xmax,exp_distributions = set_Xmax_distr()
meanLgE = (xmax['maxlgE']+xmax['minlgE'])/2
    

In [ ]:
A_tot, frac = mass.fraction_Atmosphere(Tensor, E_times_k, A, Z, w_zR, xmax) # computing mass fractions on the top of the atmosphere
A_new, f_new = mass.reduced_fractions(A_tot, frac,len(meanLgE)) # reducing in number

In [ ]:
xMax_limits = Table.read('../Data/Range.txt', format='ascii.basic', delimiter=" ", guess=False) # read the limits of xmax distribution

In [ ]:
gumbel = Convolve_all(xmax,A_new, exp_distributions, xMax_limits, model) # compute all the gumbel functions (for each energy and mass)


In [ ]:
group = Print_Xmax_group(A,A_new,f_new,xmax,xMax_limits,exp_distributions, model) # compute gumbel grouping the mass fractions (useful just for the flot)


In [ ]:
    start = time.time()
    fig = plt.figure()

    for i in range( len(meanLgE)): # loop omn energy (try to avoid)

        sum = np.zeros((len(A_new),len(exp_distributions[i][xMax_limits['Min'][i]:xMax_limits['Max'][i]])))

        f_new[i] = f_new[i]/np.sum (f_new[i]) # normalizing the mass fractions

        for j in range(len(A_new)):
            sum[j] = np.multiply(gumbel[i][j], f_new[i][j]) # weighting the mass fractions

        final = np.sum(sum, axis = 0)/np.sum(f_new[i])

        data = exp_distributions[i][xMax_limits['Min'][i]:xMax_limits['Max'][i]] 
        Dev = deviance_Xmax_distr(data,final, xmax['nEntries'][i]) # compute the deviance, between the data and the expected distribution
        print(meanLgE[i], " ", Dev)
        
        ax1 = fig.add_subplot(5, 5, i+1) ## from hereinafter just plot stuff
        plt.xlim(600,1000)
        data = data/np.sum(data)
        x = np.arange(0,2000,20)
        x = x[xMax_limits['Min'][i]:xMax_limits['Max'][i]]
        plt.errorbar(x, data/np.sum(data), fmt='o', color = 'black')

        plt.text(900, 0.25, meanLgE[i] , fontsize=6, color = 'black',fontweight='bold')
   
        plt.plot(x, final, linewidth=2, linestyle='--', color ='brown')
        [plt.plot(x, group[i][h], linewidth=2, linestyle='-', color =constant.colors[h]) for h in range(len(A))]

    end = time.time()
    print("Elapsed time_3 ", end - start)
    #plt.tight_layout()
    plt.show()